### 决策树

决策树，又称为"判定树"，是基于树结构来进行决策。决策树学习的目的是为
了产生一棵泛化能力强，即处理未见示例能力强的决策树。决策树学习基本算法过程如下:

![avatar](../image/机器学习_决策树_图1.jpg)

显然, 决策树的生成是一个递归过程. 在决策树基本算法中, 有三种情形会
导致递归返回:
* 当前结点包含的样本全属于同一类别, 无需划分;
* 当前属性集为空, 或是所有样本在所有属性上取值相同, 无法划分;
* 当前结点包含的样本集合为空, 不能划分。

**决策树学习的关键在于如何选择最优划分属性**。

### 划分选择

我们希望决策树的分支结点所包含的样本尽可能属于同一类别，即节点的"纯度"越来越高。

#### 信息增益
“信息嫡” (information entropy)是度量样本集合纯度最常用的一种指标。
假定当前样本集合 D 中第 $k$ 类样本所占的比例为 $p_{k}(k=1,2, \ldots,|\mathcal{Y}|),$ 则 $D$ 的信息嫡定义为
$$
\operatorname{Ent}(D)=-\sum_{k=1}^{|\mathcal{Y}|} p_{k} \log _{2} p_{k}
$$
$\operatorname{Ent}(D)$ 的值越小, 则 $D$ 的纯度越高.

假定离散属性 $a$ 有 $V$ 个可能的取值 $\left\{a^{1}, a^{2}, \ldots, a^{V}\right\},$ 若使用 $a$ 来对样本集
$D$ 进行划分，则会产生 $V$ 个分支结点, 其中第 $v$ 个分支结点包含了 $D$ 中所有在 属性 $a$ 上取值为 $a^{v}$ 的样本, 记为 $D^{v}$. 我们可根据式(4.1) 计算出 $D^{v}$ 的信息嫡, 再考虑到不同的分支结点所包含的样本数不同, 给分支结点赋予权重 | $D^{v}|/| D |$ 即样本数越多的分支结点的影响越大, 于是可计算出用属性 $a$ 对样本集 D 进行 划分所获得的“信息增益” (information gain)
$$
\operatorname{Gain}(D, a)=\operatorname{Ent}(D)-\sum_{v=1}^{V} \frac{\left|D^{v}\right|}{|D|} \operatorname{Ent}\left(D^{v}\right)
$$
一般而言，信息增益越大, 则意味着使用属性 $a$ 来进行划分所获得的“纯 度提升”越大. 因此, 我们可用信息增益来进行决策树的划分属性选择, 即在选择最优划分属性时选择属性$a_{*}=\underset{a \in A}{\arg \max } \operatorname{Gain}(D, a)$。

实际上, **信息增益准则对可取值数目较多的属性有所偏好**，为減少这种
偏好可能带来的不利影响, 著名的C4.5决策树算法不直接使 用信息增益，而是使用“增益率” (gain ratio) 来选择最优划分属性. 增益率定义为
$$
\operatorname{Gain}_{ratio}(D, a)=\frac{\operatorname{Gain}(D, a)}{\operatorname{IV}(a)}
$$
其中
$$
\mathrm{IV}(a)=-\sum_{v=1}^{V} \frac{\left|D^{v}\right|}{|D|} \log _{2} \frac{\left|D^{v}\right|}{|D|}
$$
称为属性a的"固有值"(intrinsic value).属性a的可能取值数目越多(即 V 越大), 则 IV $(a)$ 的值通常会越大. 

需注意的是, **增益率准则对可取值数目较少的属性有所偏好**, 因此, C4.5算法并不是直接选择增益率最大的候选划分属性, 而是使用了一个启发式
:先从候选划分属性中找出信息增益高于平均水平的属性, 再从中选择增益率最高的。

#### 基尼指数
CART 决策树使用“基尼指数” (Gini index)来选
择划分属性.数据集 D 的纯度可用基尼值来度量:
$$
\begin{aligned}
\operatorname{Gini}(D) &=\sum_{k=1}^{|\mathcal{Y}|} \sum_{k^{\prime} \neq k} p_{k} p_{k^{\prime}} \\
&=1-\sum_{k=1}^{|\mathcal{Y}|} p_{k}^{2}
\end{aligned}
$$
直观来说，Gini(D) 反映了从数据集 D 中随机抽取两个样本，其类别标记
不一致的概率. 因此, Gini(D) 越小，则数据集 D 的纯度越高。
属性 a 的基尼指数定义为
$$
\operatorname{Gini}_{\text {index }}(D, a)=\sum_{v=1}^{V} \frac{\left|D^{v}\right|}{|D|} \operatorname{Gini}\left(D^{v}\right)
$$
于是, 我们在候选属性集合 A 中，选择那个使得划分后基尼指数最小的属
性作为最优划分属性, 即 $a_{*}=\underset{a \in A}{\arg \min } \operatorname{Gini}_{index} (D, a)$

### 剪枝处理

剪枝是决策树学习算法对付“过拟合”的主要手段，决策树剪枝的基本策略有“预剪枝”和“后剪枝”。

预剪枝是指在决策树生成过程中, 对每个结点在划前先进行估计, 若当前结点的划分不能带来决策树泛化性能提升, 则停止划分并将当前结点标记为叶结点; 

后剪枝则是先从训练集生成一棵完整的决策树, 然后自底向上地对非叶结点进行考察, 若将该结点对应的子树替换为叶结点能带来决策树泛化性能提升, 则将该子树替换为叶结点。

我们可以预留一部分数据用作“验证集”以进行泛化性能的评估。

一般情形下, 后剪枝决策树的欠拟合风险很小，泛化性能往往优于预剪枝决策树。但后剪枝过程是在生成完全决策树之后进行的, 并且要自底向上地对树中的所有非叶结点进行逐一考察, 因此其训练时间开销比未剪枝决策树和预剪枝决策树都要大得多。

### 连续值处理

前面我们讨论的都是基于离散属性来生成决策树，现实学习任务中常会遇到连续属性。此时采用,**连续属性离散化技术**可排上用场，最简单的策略是采用二分法对连续属性进行处理，这正是C4.5决策树算法中采用的机制。

给定样本集 D 和连续属性 $a,$ 假定 $a$ 在 $D$ 上出现了 $n$ 个不同的取值, 将这 些值从小到大进行排序, 记为 $\left\{a^{1}, a^{2}, \ldots, a^{n}\right\} .$ 基于划分点 $t$ 可将 $D$ 分为子集 $D_{t}^{-}$ 和 $D_{t}^{+},$ 其中 $D_{t}^{-}$ 包含那些在属性 $a$ 上取值不大于 $t$ 的样本, 而 $D_{t}^{+}$ 则包含 那些在属性 $a$ 上取值大于 $t$ 的样本. 显然, 对相邻的属性取值 $a^{i}$ 与 $a^{i+1}$ 来说, $t$在区间 $\left[a^{i}, a^{i+1}\right)$ 中取任意值所产生的划分结果相同. 因此, 对连续属性 $a,$ 我们 可考察包含 $n-1$ 个元素的候选划分点集合
$$
T_{a}=\left\{\frac{a^{i}+a^{i+1}}{2} | 1 \leqslant i \leqslant n-1\right\}
$$
然后，我们就可像离散属性值一样来考察这些划分点，此时信息增益为:
$$
\begin{aligned}
\operatorname{Gain}(D, a) &=\max _{t \in T_{a}} \operatorname{Gain}(D, a, t) \\
&=\max _{t \in T_{a}} \operatorname{Ent}(D)-\sum_{\lambda \in\{-,+\}} \frac{\left|D_{t}^{\lambda}\right|}{|D|} \operatorname{Ent}\left(D_{t}^{\lambda}\right)
\end{aligned}
$$

具体计算可参考[决策树-连续值处理](https://blog.csdn.net/u012328159/article/details/79396893)

需要注意的是，与离散属性不同，若当前节点划分属性为连续属性，**该属性还可作为其后代节点的划分属性。**

### 缺少值处理

在实际中部分样本会不完整，若完全放弃不完整样本会导致信息的浪费，所有有必要考虑利用有缺少属性值的训练样例来进行学习。

我们需解决两个问题: (1)如何在属性值缺失的情况下进行划分属性选择?
(2)给定划分属性,若样本在该属性上的值缺失, 如何对样本进行划分?

给定训练集 D 和属性 $a,$ 令 $\tilde{D}$ 表示 $D$ 中在属性 $a$ 上没有缺失值的样本子
集. 对问题(1)，显然我们仅可根据 $\tilde{D}$ 来判断属性 $a$ 的优劣. 假定属性 $a$ 有 $V$ 个 可取值 $\left\{a^{1}, a^{2}, \ldots, a^{V}\right\},$ 令 $\tilde{D}^{v}$ 表示 $\tilde{D}$ 中在属性 $a$ 上取值为 $a^{v}$ 的样本子集, $\tilde{D}_{k}$ 表示 $\tilde{D}$ 中属于第 $k$ 类 $(k=1,2, \ldots,|\mathcal{Y}|)$ 的样本子集, 则题然有 $\tilde{D}=\bigcup_{k=1}^{|\mathcal{Y}|} \tilde{D}_{k}$
$\tilde{D}=\bigcup_{v=1}^{V} \tilde{D}^{v} .$ 假定我们为每个样本 $\boldsymbol{x}$ 赋予一个权重 $w_{x},$ 并定义

$$
\begin{aligned}
\rho &=\frac{\sum_{\boldsymbol{x} \in \tilde{D}} w_{\boldsymbol{x}}}{\sum_{\boldsymbol{x} \in D} w_{\boldsymbol{x}}} \\
\tilde{p}_{k} &=\frac{\sum_{\boldsymbol{x} \in \tilde{D}_{k}} w_{\boldsymbol{x}}}{\sum_{\boldsymbol{x} \in \tilde{D}} w_{\boldsymbol{x}}} \quad(1 \leqslant k \leqslant|\mathcal{Y}|) \\
\tilde{r}_{v} &=\frac{\sum_{\boldsymbol{x} \in \tilde{D}^{v}} w_{\boldsymbol{x}}}{\sum_{\boldsymbol{x} \in \tilde{D}} w_{\boldsymbol{x}}} \quad(1 \leqslant v \leqslant V)
\end{aligned}
$$
直观地看, 对属性 $a, \rho$ 表示无缺失值样本所占的比例, $\tilde{p}_{k}$ 表示无缺失值样本中 第 $k$ 类所占的比例, $\tilde{r}_{v}$ 则表示无缺失值样本中在属性 $a$ 上取值 $a^{v}$ 的样本所占 的比例. 显然, $\sum_{k=1}^{|\mathcal{Y}|} \tilde{p}_{k}=1, \sum_{v=1}^{V} \tilde{r}_{v}=1$。

信息增益的计算式可推广为:
$$
\begin{aligned}
\operatorname{Gain}(D, a) &=\rho \times \operatorname{Gain}(\tilde{D}, a) \\
&=\rho \times\left(\operatorname{Ent}(\tilde{D})-\sum_{v=1}^{V} \tilde{r}_{v} \operatorname{Ent}\left(\tilde{D}^{v}\right)\right)
\end{aligned}
$$
其中
$$
\operatorname{Ent}(\tilde{D})=-\sum_{k=1}^{| \mathcal{Y |}} \tilde{p}_{k} \log _{2} \tilde{p}_{k}
$$

对问题(2), 若样本 $\boldsymbol{x}$ 在划分属性 $a$ 上的取值已知, 则将 $\boldsymbol{x}$ 划入与其取值对 应的子结点，且样本权值在子结点中保持为 $w_{x} .$ 若样本 $\boldsymbol{x}$ 在划分属性 $a$ 上的取 值未知, 则将 $\boldsymbol{x}$ 同时划入所有子结点, 且样本权值在与属性值 $a^{v}$ 对应的子结点 中调整为 $\tilde{r}_{v} \cdot w_{\boldsymbol{x}} ;$ 直观地看, 这就是让同一个样本以不同的概率划入到不同的子结点中去。

**在决策树学习开始阶段，根结点中各样本的权重初始化为1.**

具体例子可参考周志华《机器学习》P87~P88部分。